# Structured Outputs
One of the most powerful features of LLMs is the ability to produce output which conforms to a pre-determined structure.

We define output structures using custom Pydantic types with attribute descriptions and validation logics. See more about using Pydantic types for structured outputs using langchain [here](https://python.langchain.com/docs/concepts/structured_outputs/). This package builds on that functionality by making it easy to request structured output at any point in an exchange.

In [1]:
import pydantic

import sys
sys.path.append('../src/')

import simplechatbot
from simplechatbot.openai_chatbot import OpenAIChatBot

In [2]:

# optional: use this to grab keys from a json file rather than setting system variables
keychain = simplechatbot.APIKeyChain.from_json_file('../keys.json')

chatbot = OpenAIChatBot.new('gpt-4o-mini', api_key=keychain['openai'])
print(chatbot)

OpenAIChatBot(model_type=ChatOpenAI, model_name="gpt-4o-mini", tools=ToolLookup(tools={}))


In [3]:
chatbot.history

[]

In [4]:
chatbot.stream('Hello, how are you? My name is Devin. Don\'t forget it!').print_and_collect()

Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?

ChatResult(content=Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?, tool_calls=[])

In [5]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?


In [ ]:
class ResponseFormatter(pydantic.BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = pydantic.Field(description="The answer to the user's question.")
    followup_question: str = pydantic.Field(description="A follow-up question the user could ask.")

chatbot.chat_structured('what is your favorite cat?', output_structure=ResponseFormatter)

StructuredOutputResult(data=answer="As an AI, I don't have personal feelings or favorites, but many people love breeds like the Maine Coon for their size and friendly nature, or the Siamese for their striking appearance and vocal personality. Do you have a favorite cat breed?" followup_question='What do you like most about cats?')

In [7]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?
Human: what is your favorite cat?
AI: {"answer":"As an AI, I don't have personal feelings or favorites, but many people love breeds like the Maine Coon for their size and friendly nature, or the Siamese for their striking appearance and vocal personality. Do you have a favorite cat breed?","followup_question":"What do you like most about cats?"}


In [8]:
chatbot.stream('I like that they are so cute!').print_and_collect()

They really are cute! Their playful antics and unique personalities make them endearing companions. Do you have a cat of your own, or is there a specific breed you find the cutest?

ChatResult(content=They really are cute! Their playful antics and unique personalities make them endearing companions. Do you have a cat of your own, or is there a specific breed you find the cutest?, tool_calls=[])

In [9]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?
Human: what is your favorite cat?
AI: {"answer":"As an AI, I don't have personal feelings or favorites, but many people love breeds like the Maine Coon for their size and friendly nature, or the Siamese for their striking appearance and vocal personality. Do you have a favorite cat breed?","followup_question":"What do you like most about cats?"}
Human: I like that they are so cute!
AI: They really are cute! Their playful antics and unique personalities make them endearing companions. Do you have a cat of your own, or is there a specific breed you find the cutest?


In [ ]:
chatbot.chat_structured('what is your favorite cat?', output_structure=ResponseFormatter, add_to_history=False)

StructuredOutputResult(data=answer="I don't have personal preferences, but many people adore the Persian cat for its fluffy coat and gentle demeanor. Others might prefer playful breeds like the Bengal or the Ragdoll for their engaging personalities. What's your favorite trait in a cat?" followup_question='Do you prefer playful cats or more laid-back ones?')

In [11]:
print(chatbot.history.get_buffer_string())

Human: Hello, how are you? My name is Devin. Don't forget it!
AI: Hello, Devin! I'm here to help you with anything you need. What can I assist you with today?
Human: what is your favorite cat?
AI: {"answer":"As an AI, I don't have personal feelings or favorites, but many people love breeds like the Maine Coon for their size and friendly nature, or the Siamese for their striking appearance and vocal personality. Do you have a favorite cat breed?","followup_question":"What do you like most about cats?"}
Human: I like that they are so cute!
AI: They really are cute! Their playful antics and unique personalities make them endearing companions. Do you have a cat of your own, or is there a specific breed you find the cutest?
